# MoSDeF - A Molecular Simulation Design Framework

## Overview

[MoSDeF](mosdef.org) consists of two core Python packages, [mBuild](https://github.com/mosdef-hub/mbuild) and [Foyer](https://github.com/mosdef-hub/foyer), that, when combined with tools for workflow and analysis management (such as [Signac and Signac-Flow](https://glotzerlab.engin.umich.edu/signac/)) provide the means to perform complex molecular simulations in a **reproducible** manner. Reproducibility in this case is achieved by making all aspects of the simulation (system initialization, simulation execution, and analysis) scriptable, such that other researchers could execute your same scripts to achieve the same results. MoSDeF is also designed such that systems can be generated in a programmatic manner, facilitating screening of large structural/chemical parameter spaces.

In this overview, we will be focusing specifically on the tools we've developed to address the issue of **_system initialization_**, including the creation of a molecular model and the application of a force field (atom-typing and parameter assignment). The two tools contained within MoSDeF to address system initialization are:

  - [**mBuild**](https://github.com/mosdef-hub/mbuild): A hierarchical, component-based molecule builder
  
  - [**Foyer**](https://github.com/mosdef-hub/foyer): A package for atom-typing as well as applying and disseminating forcefields

This overview is designed to introduce you to these tools in a general manner; however, more in-depth tutorials are also available from within the [mosdef_tutorials repository](https://github.com/mosdef-hub/mosdef_tutorials).

---

---

### mBuild Units

mBuild implicitly uses the below unit system:

**Length**
* nanometers [nm]

**Angles**
* Radians for all `Compound` operations
* Degrees when building `Lattices`

### Importing mBuild

To begin, we need to import the `mbuild` package, here using the alias `mb`. This will give us access to all of the data structures and functions within mBuild. You can check to make sure the installed version is 0.10.1.

In [ ]:
import mbuild as mb
print(mb.version)

### The `Compound` class

The building block of mBuild is the `Compound` class. A `Compound` can represent anything in the hierarchy of a chemical topology: an atom, a non-atomistic bead, a molecule, a chemical moeity, a section of a molecule, a collection of molecules, and even the entire system. Each level of the hierarchy inherits from the `Compound` class. This means that `Compounds` may contain other `Compounds`, and that the same methods and attributes are present for molecule components at any level of the hierarchy.

<img src="./utils/hierarchical_design_image.png" alt="Drawing" style="width: 700px;"/>

Let's make a compound by creating an instance of the `Compound` class. The default arguments are fine for now, so we don't need to specify anything.

In [ ]:
mycompound = mb.Compound()
mycompound

The `Compound` class has a `visualize` method that enables in-notebook rendering.

In [ ]:
mycompound.visualize()

Now that we've built a single-particle system, let's look at how we can build molecular structures. Compounds can be created by assembling systems piece-by-piece from constituent particles; however, it is typically more practical to load coordinates and bonds from a structure file (such as PDB, GRO, MOL2, XYZ, etc.).

First we will build a linear alkane chain using each approach.

---

## Creating an Alkane from CH2 building blocks

In our first approach for creating an alkane we will set-up routines for connecting CH2 building blocks (and we will cap the ends of our chain with hydrogens).

### Loading from a PDB structure file

First, we'll load a CH2 moiety into an mBuild `Compound` by reading from a PDB structure file (created using [Avogadro](https://avogadro.cc/)). This will create an mBuild `Compound` containing three atoms (C, H, H), as well as two C-H bonds. The `visualize` method allows us to view our `Compound` directly within the notebook. This visualization is provided by [`nglview`](https://github.com/arose/nglview).

In [ ]:
ch2 = mb.load('utils/ch2.pdb')
ch2.visualize()

Note, formats such as PDB include bonding information.  One could presumably load other formats without bonding information, and specify these bonds manually.  Additionally, one can explicitly define atom locations and bonds; for example, see  [mBuild Tutorial 01: Basic Functionality](https://github.com/mosdef-hub/mbuild_tutorials/blob/master/mBuild_01_Basic_Functionality.ipynb).

### `Ports` and `Compound` classes

In order to connect `Compounds` we need to define locations where bonds can be formed (we can also add bonds manually through the `add_bond` method, which we will not cover here). mBuild handles this by allowing users to define `Ports` on particles, which essentially act as dangling bonds.

However, if one had to re-write the commands for loading a CH2 molecule and adding `Ports` each time they wanted to create a molecule that included a CH2 unit, the process would be quite cumbersome. Instead, we can create a reusable class that defines our CH2 `Compound`. This approach allows one to encapsulate the routines for creating a molecular moiety into an object that can be instantiated and in a manner that can easily be shared with others.

Below is a class definition for a CH2 moiety that uses the _same_ command we used above to load coordinates and bonds from a PDB structure file and features a few lines that add `Ports` to the carbon atom.

For additional information, see [mBuild Tutorial 02: Reusing Components](https://github.com/mosdef-hub/mbuild_tutorials/blob/master/mBuild_02_Reusing_Components.ipynb).

In [ ]:
class CH2(mb.Compound):
    def __init__(self):
        super(CH2, self).__init__()
        mb.load('utils/ch2.pdb', compound=self)
        carbon = list(self.particles_by_name('C'))[0]
        self.add(mb.Port(anchor=carbon, orientation=[0, 1, 0], separation=0.075), 'up')
        self.add(mb.Port(anchor=carbon, orientation=[0, -1, 0], separation=0.075), 'down')

If we instantiate this class and visualize we should see the same result we obtained earlier. We can pass the `show_ports=True` argument to `visualize` to see the `Ports` we've added to the carbon atom.

In [ ]:
ch2 = CH2()
ch2.visualize(show_ports=True)

### Examining the `Compound` data structure

Now that we have created a `Compound` we can examine the contents.  For example, simply calling the `Compound` will provide us with a summary of the contents.

In [ ]:
# simply call the compound to print a summary of the number particles and bonds
ch2.save('ch2.xyz', overwite=True)


In [ ]:
ch2_xyz = mb.load('./ch2.xyz')

We can examine the coordinates in multiple ways as shown below:

In [ ]:
# view the coordinates of the atoms in the compound
ch2.xyz

In [ ]:
# use the list function to iterate over the atoms and their positions in the compound
list(ch2)

To view the bonds, we can call the `bonds` method as part of the `Compound` taking advantage of `list`:

In [ ]:
#list the pairs of atoms that are bonded; each pair appears between parantheses, i.e., (atom1, atom2)
list(ch2.bonds())

In [ ]:
# we can also format the output of bonds to simply list the pairs of bonded atoms by name alone
for pair in ch2.bonds():
    print(pair[0].name, '-', pair[1].name)

# equivalent shorthand output using list comprehension
['{}-{}'.format(pair[0].name, pair[1].name) for pair in ch2.bonds()]

We can also view a summary of the  ports associated with a `Compound`:

In [ ]:
ch2.all_ports()

### More complex classes. Connecting CH2 moieties into an alkane

Now we'll create a more complex class that defines the instructions for connecting CH2 moieties into a linear alkane chain.

The code below instantiates CH2 moieties inside of a "for" loop, where the number of iterations is dependent on the desired length of the chain. The length of the chain can be toggled through the `chain_length` argument provided to the class constructor. We also import a hydrogen `Compound` from mBuild's `atoms` library to cap the ends of our chain.

This is shown pictorially below.
<img src="./utils/figure_connecting.png" alt="Drawing" style="width: 700px;"/>

**Note:** For this general overview, we do not intend for users (particularly those new to Python and object-oriented programming) to get too bogged down in the syntax. Instead, the emphasis should be that with mBuild we can encapsulate a series of routines (a "recipe") into a class, and that these routines can be defined in a manner that gives the class structural/chemical flexibility.

For additional examples, see tutorials: 
- [mBuild Tutorial 03: Connecting Components with Ports](https://github.com/mosdef-hub/mbuild_tutorials/blob/master/mBuild_03_Connecting_Components_with_Ports.ipynb) 
- [mBuild Tutorial 04: Constructing Larger Compounds](https://github.com/mosdef-hub/mbuild_tutorials/blob/master/mBuild_04_Constructing_Larger_Compounds.ipynb)
- [mBuild Tutorial 05: Creating Flexible Classes](https://github.com/mosdef-hub/mbuild_tutorials/blob/master/mBuild_05_Creating_Flexible_Classes.ipynb)

In [ ]:
from mbuild.lib.atoms import H

class Alkane(mb.Compound):
    def __init__(self, chain_length=1):
        super(Alkane, self).__init__()
        last_monomer = CH2()
        hydrogen = H()
        mb.force_overlap(move_this=hydrogen,
                         from_positions=hydrogen['up'],
                         to_positions=last_monomer['up'])
        self.add(last_monomer)
        self.add(hydrogen)
        for _ in range(chain_length-1):
            current_monomer = CH2()
            mb.force_overlap(move_this=current_monomer,
                             from_positions=current_monomer['up'],
                             to_positions=last_monomer['down'])
            self.add(current_monomer)
            last_monomer=current_monomer
        hydrogen = H()
        mb.force_overlap(move_this=hydrogen,
                         from_positions=hydrogen['up'],
                         to_positions=last_monomer['down'])
        self.add(hydrogen)

Because we've defined our class to take `chain_length` as an argument, we can toggle the chemistry of our system (in this case the number of carbons in a linear alkane) by changing the value we provide for this argument upon instantiation.

For example, let's create a butane molecule.

In [ ]:
butane = Alkane(chain_length=4)
butane.visualize()

The geometry of this molecule is not entirely realistic as all backbone atoms featuring 180° angles in the alkane molecules, all hydrogen atoms in plane. This can be addressed by placing Particles and Ports in more realistic locations, either manually or by using energy minimized inputs.

Alternatively, a `Compound` can be constructed and then energy minimized, either through a simulation engine or using the energy_minimization method in `mBuild`, which uses the [Open Babel toolkit](http://openbabel.org/dev-api/). See tutorial [mBuild Tutorial 07: Energy Minimization](https://github.com/mosdef-hub/mbuild_tutorials/blob/master/mBuild_07_Energy_Minimization.ipynb) for more information about the use of this method.

In [ ]:
butane.energy_minimization()

Now let's change the value of `chain_length` to create a decane.

In [ ]:
decane = Alkane(chain_length=10)
decane.visualize()

## Creating an Alkane by importing a pre-defined class


In addition to allowing chemistry to be easily tuned through the presence of arguments, by encapsulating `Compound` recipes into a class, we then never have to define it again(!) and can easily share our recipe with other researchers. In other words, if Researcher A at University Y creates a class defining routines for generating some complex molecular system, Researcher B at University Z can simply import this class and use it. We are currently at work towards the implementation of a plugin architecture for mBuild that will make it easier for researchers to disseminate the `Compound` classes they've created.

For example, an `Alkane` class already exists within the `utils` directory of this repository. We can import that class and in two lines of code (excluding the visualization) we'll be able to construct a linear alkane of arbitrary length.

In [ ]:
from utils import Alkane
octane = Alkane(chain_length=8)
octane.visualize()

One immediate concern you may have is that many of these molecules we've generated feature non-realistic configurations (such as the alkane above with all angles at 180 degrees). However, this can typically be resolved through an energy minimization of the system you create in whichever simulation package you like to use. Additionally, mBuild also offers a built-in energy minimization routine (built around OpenBabel and OpenMM) that will attempt to optimize a `Compound`'s geometry.

## Altering `Compounds`. Creating an alcohol.


mBuild contains routines for the addition and removal of particles. Here, we'll explore this functionality by changing our hexane molecule into _hexanol_.
Note, we could do this by manually changing the class itself, or simply by removing the temrinal hydration and adding a hydroxyl in its place. 

First, we'll define a class for a hydroxyl group featuring a single `Port` on the oxygen to represent the dangling bond.

We'll also go ahead and instantiate this class and visualize the resulting `Compound`.

In [ ]:
class OH(mb.Compound):
    def __init__(self):
        super(OH, self).__init__()
        self.add(mb.Particle(name='O', pos=[0.0, 0.0, 0.0]), label='O')
        self.add(mb.Particle(name='H', pos=[0.0, 0.1, 0.0]), label='H')
        self.add_bond((self['O'], self['H']))
        self.add(mb.Port(anchor=self['O'], orientation=[0, -1, 0], separation=0.075), label='down')
        
hydroxyl = OH()
hydroxyl.visualize(show_ports=True)

In this example, we will use the `label` assigned to the up cap to differentiate it from other hydogen atoms in the system. This label can be determined by examining the class source code itself included in util, or by quering the `Compound` instance itself. Below we can see that we have 3 labels, chain, 'up_cap', and 'down_cap'. 

In [ ]:
list(octane.labels)

Now we'll remove a hydrogen from one end of our octane. This will create a `Port` in it's place, representing the dangling bond on the carbon.

In the class definition, the hydrogen atom we will remove was provided with a label, `up_cap`. We can use this label to refer to this hydrogen, and remove it.

In [ ]:
octane.remove(octane['up_cap'])
octane.visualize(show_ports=True)

Now we can use the `force_overlap` method to attach the hydroxyl cap to the octane with the dangling bond.

In [ ]:
available_ports = octane.all_ports()
mb.force_overlap(move_this=hydroxyl,
                 from_positions=hydroxyl['down'],
                 to_positions=available_ports[0])
octanol = mb.Compound(name='Octanol')
octanol.add(octane, label='alkane')
octanol.add(hydroxyl, label='hydroxyl')
octanol.visualize()

## Setting up bulk systems


Typically we aren't desiring to run simulations of a single molecule. Fortunately, mBuild offers several routines to help create more complex systems. 

For example, mBuild provides users with an interface to [PACKMOL](http://m3g.iqm.unicamp.br/packmol/home.shtml) to set up bulk systems through the `fill_box` function. Here we'll use `fill_box` to place ten octanol molecules into a 3nm x 3nm x 3nm box. We can provide a seed for PACKMOL's random number generator to ensure the configuration is reproducible.

For additional information, see [mBuild Tutorial 06: Setting Up Bulk Systems](https://github.com/mosdef-hub/mbuild_tutorials/blob/master/mBuild_06_Setting_Up_Bulk_Systems.ipynb).


In [ ]:
octanol_box = mb.fill_box(octanol, n_compounds=10, box=[3, 3, 3], seed=2)
octanol_box.visualize()

## Surface functionalization

mBuild also provides routines for functionalization surfaces. There are a few surfaces available within mBuild's `surfaces` library; however, in the future we are hoping to feature a more comprehensive `surfaces` plugin.

Here, we'll load a surface of $\beta$-cristobalite silica.

**Note:** Harmless warning messages are currently generated by one of the packages mBuild depends on. To reduce clutter, we are filtering those here, so you can safely ignore the warnings filter.

For additional information, see [mBuild Tutorial 09: Surface Functionalization](https://github.com/mosdef-hub/mbuild_tutorials/blob/master/mBuild_09_Surface_Functionalization.ipynb).

In [ ]:
import warnings
warnings.filterwarnings(action="ignore", category=FutureWarning)

from mbuild.lib.surfaces import Betacristobalite
surface = Betacristobalite()
surface.visualize(show_ports=True)

We can use the `TiledCompound` class to expand our surface if we wanted.

In [ ]:
tiled_surface = mb.recipes.TiledCompound(surface, n_tiles=(2, 1, 1))
tiled_surface.visualize(show_ports=True)

We will now remove the end-hydrogen from our octanol molecule to generate a dangling bond/`Port` that we can use to attach copies to the surface.

In [ ]:
octanol.remove(octanol['alkane']['down_cap'])
octanol.add(octanol.all_ports()[0], 'down', containment=False)
octanol.visualize(show_ports=True)

We can use mBuild's `Pattern` class to create a pattern to define the arrangement of molecules on the surface. Here, we will create a `Random2DPattern` of 50 points in *xy* space. The `apply_to_compound` method can be used to attach copies of a molecules to a surface at locations designated by the `Pattern`. We can also provide a backfill `Compound` (in this case hydrogen) to fill vacant sites.

In [ ]:
pattern = mb.Random2DPattern(10)
hydrogen = H()
chains, backfills = pattern.apply_to_compound(guest=octanol, host=tiled_surface, backfill=hydrogen)
functionalized_surface = mb.Compound(subcompounds=[tiled_surface, chains, backfills])
functionalized_surface.visualize()

Again, the above routines could be wrapped into a class. Here we'll load the `AlkaneMonolayer` class from mBuild's `examples` module. You can change `n` in the `Random2DPattern` instance to toggle the number of chains in the system. `tile_x` and `tile_y` can be toggled to expand the surface as desired, while `chain_length` can be toggled to alter the lengths of the alkylsilane chains.

In [ ]:
from mbuild.examples.alkane_monolayer.alkane_monolayer import AlkaneMonolayer
pattern = mb.pattern.Random2DPattern(n=50, seed=123)
monolayer = AlkaneMonolayer(pattern=pattern, tile_x=2, tile_y=1, chain_length=12)
monolayer.visualize()

## Applying force fields with `foyer`

In order to run a simulation of any system we have built with mBuild, we need to apply a force field and write the necessary data files. Here we will consider two of the input files needed to run a GROMACS simulation, the `.gro` and `.top` files. The `.gro` file contains atomic coordinates and box information. The `.top` file contains the relevant force field parameters. Other molecular simulation engines use different files but their contents overlap with these GROMACS files.

`Compounds` includes a `save` method that writes to many molecular simulation file formats.

We will also specify a `residues` argument. This allows us to treat every `Compound` with the name `Octanol` as a separate molecule but is not strictly necessary for writing out this file. We also typically pass the `overwrite` argument in case a file of that name already exists.

In [ ]:
octanol_box.save('system.gro', residues='Octanol', overwrite=True)

Since the `.top` file contains the force field information, we need to apply a force field to our system. Here, we will use the OPLS all-atom force field. To do this, we will use the `foyer` library, which atom-types and parametrizes systems for molecular simulation.

Foyer force fields are defined within an XML file that contains both the "rules" required for atom-typing as well as the force field parameters within a single file. 


`foyer` atom-types based on the local environment (chemical substructure) of an atom . The [SMARTS](http://www.daylight.com/dayhtml/doc/theory/theory.smarts.html) language can be used to define these chemical substructures. For example, the SMARTS string `"[C;X4](F)(F)(F)(C)"`, which prescribes the OPLS-AA atom type `opls_961`, states that this atom type should be used when:
- the atom of interest is element carbon, i.e., `[C]`
- the carbon has 4 connected neighbors, i.e., `[C;X4]`
- three neighbors are F, i.e., `(F)(F)(F)`
- one neighbor is C, i.e., `(C)`

The Foyer XML format is an extension of the [OpenMM forcefield XML format](http://docs.openmm.org/7.0.0/userguide/application.html#creating-force-fields), which is an established file format for storing force field data. `foyer` extends this format to include information necessary for atom-typing, which is the process of specifying atom types in a system according to their definitions.

The `AtomTypes` section of a `foyer` XML includes in each `Type` four additional attributes not found in OpenMM:
* `def` - SMARTS string describing the chemical substructure of this atom type (Follow [this link](https://github.com/mosdef-hub/foyer/blob/master/docs/smarts.md) for more on SMARTS-based atomtyping using Foyer.)
* `desc` - Brief description of the atom type (optional)
* `doi` - DOI reference for parameters associated with this atom type (optional)
* `overrides` - One or more atom types to 'override', providing precedence to this atom type

Let's first take a quick look at this file.

In [ ]:
!cat utils/oplsaa-alcohol.xml

With this force field XML file, `foyer` will use the SMARTS strings to atom-type our system and will then apply the proper force field parameters. We'll call the `save` method again to write a `.top` file, this time using the `forcefield_files` argument to specify that we want to use the XML we just looked at. Additionally, as OPLS-AA uses _geometric_ mixing rules as opposed to Lorentz-Berthelot, we need to pass this argument to `save` as well.

**Note:** The warning message about unparameterized impropers can be safely ignored, as OPLS-AA does not include any impropers for our system. By default, Foyer will warn the user if improper parameters are not specified for all possible impropers and will exit with an error if bond, angle, or dihedral parameters are not specified for all possible bonds, angles, dihedrals. (This behavior may be overridden if desired.)

In [ ]:
octanol_box.save('system.top', forcefield_files='utils/oplsaa-alcohol.xml', residues='Octanol',
                 combining_rule='geometric', overwrite=True)

Finally, just to check that these files were written correctly, we can look at their contents

In [ ]:
!cat system.gro

In [ ]:
!cat system.top

### Working with coarse-grained and united atom forcefields

Foyer supports SMARTS definitions involving non-atomistic components such as coarse-grained or united atom beads. Non-elemental species can easily be defined by prepending the name of custom "element" with an underscore `_`.

For example, the following lines could be used to describe beads representing `_CH2` groups in a polymer using the TraPPE forcefield. 

`  <Type name="CH2_sp3" class="CH2" element="_CH2" mass="14.02700" 
   def="[_CH2;X2]([_CH3,_CH2])[_CH3,_CH2]" 
   desc="Alkane CH2, united atom" doi="10.1021/jp972543+"/>`
  
Here, the SMARTS definition `[_CH2;X2]([_CH3,_CH2])[_CH3,_CH2]` states that for atom-type `CH2_sp3`
- the bead of interest is `[_CH2]`
- it has 2 connected neighbors, i.e., `[_CH2;X2]`
- those neighbors can be either `_CH2` or `_CH3`, i.e., `([_CH3,_CH2])[_CH3,_CH2]`

This uses the "or" operator in SMARTS, which is a comma. In this case, these neighbors can be _either_ `_CH2` or `_CH3`.


For more information on nano-atomistic forcefields, see [Foyer Tutorial 02: SMARTS for Non-Atomistic Systems](https://github.com/mosdef-hub/foyer_tutorials/blob/master/Foyer_02_SMARTS_for_Non-Atomistic_Systems.ipynb).

For additional information on Foyer itself, see the [Foyer tutorial repository](https://github.com/mosdef-hub/foyer_tutorials) and [github page](https://github.com/mosdef-hub/foyer).

This concludes the general MoSDeF overview. For more in-depth tutorials into mBuild and Foyer and some of their use cases, refer to any of these links:
- [mosdef.org](mosdef.org)
- [mosdef_tutorials](https://github.com/mosdef-hub/mosdef_tutorials)
- [mbuild_tutorials](https://github.com/mosdef-hub/foyer_tutorials)
- [mosdef-workflows](https://github.com/mosdef-hub/mosdef-workflows)